# Introduction

In [1]:
import json

#########################

import ase
from ase import io  # noqa

import numpy as np

from sklearn.decomposition import PCA

from rascaline import SoapPowerSpectrum

import chemiscope

In [2]:
soap = SoapPowerSpectrum(
    cutoff=3.5,
    max_radial=5,
    max_angular=5,
    atomic_gaussian_width=0.2,
    gradients=False,
    cutoff_function={"ShiftedCosine": {"width": 0.5}},
    radial_basis={"Gto": {}},
)


frames = ase.io.read("example.xyz", ":")

descriptor = soap.compute(frames)
descriptor.densify(["species_neighbor_1", "species_neighbor_2"])

soap_vectors = descriptor.values
soap_vectors_per_structure = np.zeros((len(frames), soap_vectors.shape[1]))

start = 0
for i, frame in enumerate(frames):
    stop = start + len(frame)
    soap_vectors_per_structure[i, :] = np.sum(soap_vectors[start:stop, :], axis=0)
    start = stop

pca = PCA(n_components=2)

properties = {
    "PCA": {
        "values": pca.fit_transform(soap_vectors_per_structure),
        "target": "structure",
    }
}

In [3]:
chemiscope.display(frames, properties)

widget(value='{"meta": {"name": " "}, "structures": [{"size": 5, "names": ["C", "H", "H", "H", "H"], "x": [1.8…